In [ ]:
# Set up packages for lecture. Don't worry about understanding this code, but
# make sure to run it if you're following along.
import numpy as np
import babypandas as bpd
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats
import matplotlib.pyplot as plt
from scipy import stats
import otter
set_matplotlib_formats("svg")
plt.style.use('ggplot')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option("display.max_rows", 7)
pd.set_option("display.max_columns", 8)
pd.set_option("display.precision", 2)

# Setup to start where we left off last time
keep_cols = ['business_name', 'inspection_date', 'inspection_score', 'risk_category', 'Neighborhoods', 'Zip Codes']
restaurants_full = bpd.read_csv('data/restaurants_full.csv').get(keep_cols)
bakeries = restaurants_full[restaurants_full.get('business_name').str.lower().str.contains('bake')]
bakeries = bakeries[bakeries.get('inspection_score') >= 0] # Keeping only the rows where we know the inspection score

# Animation
from IPython.display import IFrame, display

def show_clt_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vTcJd3U1H1KoXqBFcWGKFUPjZbeW4oiNZZLCFY8jqvSDsl4L1rRTg7980nPs1TGCAecYKUZxH5MZIBh/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 509
    display(IFrame(src, width, height))

# Lecture 27 – Review, Conclusion

## DSC 10, Winter 2023

### Announcements

- The Final Exam is **tomorrow at 3PM in Galbraith Hall 242**.
    - See [this EdStem announcement](https://edstem.org/us/courses/33310/discussion/2784201) for details.
    - Assigned seats will be emailed to you by Friday.
    - We will check IDs.
    - You'll have **2 hours, 50 minutes** to work on the exam.
    - No questions during the exam.
    - The [DSC 10 Reference Sheet](https://drive.google.com/file/d/1ky0Np67HS2O4LO913P-ing97SJG0j27n/view) will be provided. No calculators or other notes.
    - Practice with old exam problems at [practice.dsc10.com](https://practice.dsc10.com).
- If at least 80% of the class fills out both [CAPEs](https://cape.ucsd.edu) and the [End of Quarter Survey](https://docs.google.com/forms/d/e/1FAIpQLSc7qjCK0DkBThgKw9SdNY9OX44SFA-UHX3MZ42dfEMcssCnWA/viewform), then the entire class gets 0.5% of extra credit on their overall grade.
    - We value your feedback!

### Agenda

- More review.
- Working on personal projects.
- Demo: Gapminder 🌎.
- Some parting thoughts.

## Bakeries 🧁

Consider this **population** of bakeries in San Francisco.

In [ ]:
bakeries

In [ ]:
score_bins = np.arange(50, 102, 2)
bakeries.plot(kind='hist', y='inspection_score', density=True, bins=score_bins, ec='w', figsize=(10, 5),
              title='Population Distribution');

For reference, the mean and standard deviation of the population distribution are calculated below.

In [ ]:
bakeries.get('inspection_score').describe()

In this case we happen to have the inspection scores for all members of the population, but in reality we won't. So let's instead take a random **sample** of 200 bakeries from the population.

In [ ]:
np.random.seed(23) # Ignore this

sample_of_bakeries = bakeries.sample(200) # SOLUTION
sample_of_bakeries

In [ ]:
sample_of_bakeries.plot(kind='hist', y='inspection_score', density=True, bins=score_bins, ec='w', figsize=(10, 5),
                        title='Sample Distribution');

Note that since we took a large, random sample of the population, we expect that our sample looks similiar to the population and has a similar mean and SD.

In [ ]:
sample_of_bakeries.get('inspection_score').describe() 

Indeed, the sample mean is quite close to the population mean, and the sample standard deviation is quite close to the population standard deviation.

Let's suppose we want to estimate the population mean (that is, the mean inspection score of all bakeries in SF).

One estimate of the population mean is the mean of our sample.

In [ ]:
sample_of_bakeries.get('inspection_score').mean()

However, our sample was random and could have been different, meaning our sample mean could also have been different.

**Question**: What's a reasonable range of possible values for the sample mean? **What is the distribution of the sample mean?**

### The Central Limit Theorem

> The Central Limit Theorem (CLT) says that the probability distribution of the **sum or mean** of a large random sample drawn with replacement will be roughly normal, regardless of the distribution of the population from which the sample is drawn.

In [ ]:
show_clt_slides()

To see an empirical distribution of the sample mean, let's take a large number of samples directly from the population and compute the mean of each one.

Remember, in real life we wouldn't be able to do this, since we wouldn't have access to the population.

In [ ]:
sample_means = np.array([])

# BEGIN SOLUTION
for i in np.arange(5000):
    sample_mean = bakeries.sample(200).get('inspection_score').mean()
    sample_means = np.append(sample_means, sample_mean)
# END SOLUTION

In [ ]:
sample_means

In [ ]:
bpd.DataFrame().assign(sample_means=sample_means).plot(kind='hist', density=True, ec='w', bins=25, figsize=(10, 5));

Unsurprisingly, the distribution of the sample mean is bell-shaped. The CLT told us that!

The CLT also tells us that

$$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$



Let's try this out.

In [ ]:
np.std(bakeries.get('inspection_score')) / np.sqrt(200)

In [ ]:
np.std(sample_means)

Pretty close! Remember that `sample_means` is an array of simulated sample means; the more samples we simulate, the closer that `np.std(sample_means)` will get to the SD described by the CLT.

Note that in practice, we won't have the SD of the population, since we'll usually just have a single sample. In such cases, we can use the SD of the sample as an estimate of the SD of the population:

In [ ]:
np.std(sample_of_bakeries.get('inspection_score')) / np.sqrt(200)

Using the CLT, we have that the distribution of the sample mean:
- is roughly normal,
- is centered at the population mean (for which the sample mean is an estimate), and
- has a standard deviation of $\frac{\text{Population SD}}{\sqrt{\text{sample size}}}$ (which can be estimated using $\frac{\text{Sample SD}}{\sqrt{\text{sample size}}}$).

Using this information, we can build a confidence interval for where we think the population mean might be. A 95% confidence interval for the population mean is given by

$$
\left[
\text{sample mean} - 2\cdot \frac{\text{sample SD}}{\sqrt{\text{sample size}}}, \
\text{sample mean} + 2\cdot \frac{\text{sample SD}}{\sqrt{\text{sample size}}}
\right]
$$

In [ ]:
sample_mean = sample_of_bakeries.get('inspection_score').mean()
sample_std = np.std(sample_of_bakeries.get('inspection_score'))

In [ ]:
[sample_mean - 2 * sample_std / np.sqrt(200), sample_mean + 2 * sample_std / np.sqrt(200)] # SOLUTION

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

Using a single sample of 200 bakeries, how can we estimate the **median** inspection score of all bakeries in San Francisco with an inspection score? What technique should we use?

A. Standard hypothesis testing

B. Permutation testing  

C. Bootstrapping

D. The Central Limit Theorem

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Bootstrapping. The CLT only applies to sample means (and sums), not to any other statistics.

</details>

There is no CLT for sample medians, so instead we'll have to resort to bootstrapping to estimate the distribution of the sample median.

Recall, bootstrapping is the act of **sampling from the original sample, with replacement**. This is also called **resampling**.

In [ ]:
# The median of our original sample – this is just one number
sample_of_bakeries.get('inspection_score').median() # SOLUTION

In [ ]:
# The median of a single bootstrap resample – this is just one number
sample_of_bakeries.sample(200, replace=True).get('inspection_score').median() # SOLUTION

Let's resample repeatedly.

In [ ]:
np.random.seed(23) # Ignore this

boot_medians = np.array([])

# BEGIN SOLUTION
for i in np.arange(5000):
    boot_median = sample_of_bakeries.sample(200, replace=True).get('inspection_score').median()
    boot_medians = np.append(boot_medians, boot_median)
# END SOLUTION

In [ ]:
boot_medians

In [ ]:
bpd.DataFrame().assign(boot_medians=boot_medians).plot(kind='hist', density=True, ec='w', bins=10, figsize=(10, 5));

Note that this distribution is not at all normal.

To compute a 95% confidence interval, we take the middle 95% of the bootstrapped medians.

In [ ]:
# BEGIN SOLUTION
left = np.percentile(boot_medians, 2.5)
right = np.percentile(boot_medians, 97.5)

[left, right]
# END SOLUTION

### Discussion Question

Which of the following interpretations of this confidence interval are valid?

1. 95% of SF bakeries have an inspection score between 85 and 88.  
2. 95% of the resamples have a median inspection score between 85 and 88.  
3. There is a 95% chance that our sample has a median inspection score between 85 and 88.  
4. There is a 95% chance that the median inspection score of all SF bakeries is between 85 and 88.  
5. If we had taken 100 samples from the same population, about 95 of these samples would have a median inspection score between 85 and 88.  
6.  If we had taken 100 samples from the same population, about 95 of the confidence intervals created would contain the median inspection score of all SF bakeries.  

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
The correct answers are Option 2 and Option 6.

</details>

## Physicians 🩺

### The setup

You work as a family physician. You collect data and you find that in 6354 patients, 3115 were children and 3239 were adults.

You want to test the following hypotheses:

- **Null Hypothesis**: Family physicians see an equal number of children and adults.
- **Alternative Hypothesis**: Family physicians see more adults than they see children.

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

Which test statistic(s) could be used for this hypothesis test? Which values of the test statistic point towards the alternative?

A. Proportion of children seen   
B. Number of children seen  
C. Number of children minus number of adults seen  
D. Absolute value of number of children minus number of adults seen

**There may be multiple correct answers; choose one.**

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
All of these but the last one would work for this alternative. Small values of these statistics would favor the alternative.
    
If the alternative was instead "Family physicians see a different number of children and adults", the last option would work while the first three wouldn't.

</details>

Let's use option B, the number of children seen, as a test statistic. Small values of this statistic favor the alternative hypothesis.

How do we generate a single value of the test statistic?

In [ ]:
np.random.multinomial(6354, [0.5, 0.5])[0] # SOLUTION

As usual, let's simulate the test statistic many, many times (10,000).

In [ ]:
test_stats = np.array([])

# BEGIN SOLUTION
for i in np.arange(10000):
    stat = np.random.multinomial(6354, [0.5, 0.5])[0]
    test_stats = np.append(test_stats, stat)
# END SOLUTION

In [ ]:
test_stats

In [ ]:
bpd.DataFrame().assign(test_stats=test_stats) \
               .plot(kind='hist', density=True, ec='w', figsize=(10, 5), bins=20);
plt.axvline(3115, lw=3, color='black', label='observed statistic')
plt.legend();

Recall that you collected data and found that in 6354 patients, 3115 were children and 3239 were adults.

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

What goes in blank (a)?

```py
p_value = np.count_nonzero(test_stats __(a)__ 3115) / 10000
```

A. `>=`

B. `>`

C. `<=`

D. `<`

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    <code><=</code>

</details>

In [ ]:
# Calculate the p-value

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

What do we do, assuming that we're using a 5% p-value cutoff?

A. Reject the null  

B. Fail to reject the null 

C. It depends

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    Fail to reject the null, since the p-value is above 0.05.

</details>

Note that while we used `np.random.multinomial` to simulate the test statistic, we could have used `np.random.choice`, too:

In [ ]:
choices = np.random.choice(['adult', 'child'], p=[0.5, 0.5], size=6354, replace=True) # SOLUTION
choices

In [ ]:
np.count_nonzero(choices == 'child') # SOLUTION

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

Is this an example of bootstrapping?

A. Yes, because we are sampling with replacement.

B. No, this is not bootstrapping.

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    No, this is not bootstrapping. Bootstrapping is when we resample from a single sample; here we're simulating data under the assumptions of a model.

</details>

## Personal projects

### Using Jupyter Notebooks after DSC 10

- You may be interested in working on data science projects of your own.
- In [this video](https://www.youtube.com/watch?v=Hq8VaNirDRQ), we show you how to make blank notebooks and upload datasets of your own to DataHub.
- Depending on the classes you're in, you may not have access to DataHub. Eventually, you'll want to install Jupyter Notebooks on your computer.
    - [Anaconda](https://docs.anaconda.com/anaconda/install/index.html) is a great way to do that, as it also installs many commonly used packages.
    - You may want to download your work from DataHub so you can refer to it after the course ends.
    - Remember, all `babypandas` code is regular `pandas` code, too!

### Finding data

These sites allow you to search for datasets (in CSV format) from a variety of different domains. Some may require you to sign up for an account; these are generally reputable sources.

Note that all of these links are also available at [rampure.org/find-datasets](https://rampure.org/find-datasets).

- [Data is Plural](https://www.data-is-plural.com/archive/)
- [FiveThirtyEight](https://data.fivethirtyeight.com/).
- [CORGIS](https://corgis-edu.github.io/corgis/csv/).
- [Kaggle Datasets](https://www.kaggle.com/datasets).
- [Google’s dataset search](http://toolbox.google.com/datasetsearch).
- [DataHub.io](https://datahub.io/collections).
- [Data.world.](https://data.world/)
- [R datasets](https://vincentarelbundock.github.io/Rdatasets/articles/data.html).
- Wikipedia. (Use [this site](https://wikitable2csv.ggor.de/) to extract and download tables as CSVs.)
- [Awesome Public Datasets GitHub repo](https://github.com/awesomedata/awesome-public-datasets).
- [Links to even more sources](https://rockcontent.com/blog/data-sources/).

### Domain-specific sources of data

- Sports: [Basketball Reference](https://www.basketball-reference.com/), [Baseball Reference](https://www.baseball-reference.com/), etc.
- US Government Sources: [census.gov](https://www.census.gov/data/tables.html), [data.gov](https://www.data.gov/), [data.ca.gov](https://data.ca.gov/), [data.sfgov.org](https://data.sfgov.org/browse?), [FBI’s Crime Data Explorer](https://crime-data-explorer.fr.cloud.gov/), [Centers for Disease Control and Prevention](https://data.cdc.gov/browse?category=NCHS).
- Global Development: [data.worldbank.org](https://data.worldbank.org/), [databank.worldbank.org](https://databank.worldbank.org/home.aspx), [WHO](https://apps.who.int/gho/data/node.home).
- Transportation: [New York Taxi trips](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), [Bureau of Transportation Statistics](https://www.transtats.bts.gov/DataIndex.asp), [SFO Air Traffic Statistics](https://www.flysfo.com/media/facts-statistics/air-traffic-statistics).
- Music: [Spotify Charts](https://spotifycharts.com/regional).
- COVID: [Johns Hopkins](https://github.com/CSSEGISandData/COVID-19).
- Any Google Forms survey you’ve administered! (Go to the results spreadsheet, then go to “File > Download > Comma-separated values”.)

Tip: if a site only allows you to download a file as an Excel file, not a CSV file, you can download it, open it in a spreadsheet viewer (Excel, Numbers, Google Sheets), and export it to a CSV.

### Join a DS3 Project Group 🤝

The Data Science Student Society organizes project groups, which are a great way to get experience and build your resume. Keep your eye out for applications! 
 - Students have the opportunity to join a team to pursue a unique data science project that will last two quarters. 
 - At the end of the project, teams will have developed a polished, complete personal project which they will showcase to their peers, faculty, and companies in the data science industry. 
 - Great for if you don't yet have a lot of experience and are looking for that first data science project to get you started.

## Demo: Gapminder 🌎

### `plotly`

- All of the visualizations (scatter plots, histograms, etc.) in this course were created using a library called `matplotlib`.
    - This library was called under-the-hood everytime we wrote `df.plot`.
- `plotly` is a different visualization library that allows us to create **interactive** visualizations.
- You may learn about it in a future course, but we'll briefly show you some cool visualizations you can make with it.

In [ ]:
import plotly.express as px

### Gapminder dataset

> Gapminder Foundation is a non-profit venture registered in Stockholm, Sweden, that promotes sustainable global development and achievement of the United Nations Millennium Development Goals by increased use and understanding of statistics and other information about social, economic and environmental development at local, national and global levels. - [Gapminder Wikipedia](https://en.wikipedia.org/wiki/Gapminder_Foundation)

In [ ]:
gapminder = px.data.gapminder()
gapminder

The dataset contains information for each country for several different years.

In [ ]:
gapminder.get('year').unique()

Let's start by just looking at 2007 data (the most recent year in the dataset).

In [ ]:
gapminder_2007 = gapminder[gapminder.get('year') == 2007]
gapminder_2007

### Scatter plot

We can plot life expectancy vs. GDP per capita. If you hover over a point, you will see the name of the country.

In [ ]:
px.scatter(gapminder_2007, x='gdpPercap', y='lifeExp', hover_name='country')

In future courses, you'll learn about transformations. Here, we'll apply a log transformation to the x-axis to make the plot look a little more linear.

In [ ]:
px.scatter(gapminder_2007, x='gdpPercap', y='lifeExp', log_x=True, hover_name='country')

### Animated scatter plot

We can take things one step further.

In [ ]:
px.scatter(gapminder,
           x = 'gdpPercap',
           y = 'lifeExp', 
           hover_name = 'country',
           color = 'continent',
           size = 'pop',
           size_max = 60,
           log_x = True,
           range_y = [30, 90],
           animation_frame = 'year',
           title = 'Life Expectancy, GDP Per Capita, and Population over Time'
          )

Watch [this video](https://www.youtube.com/watch?v=jbkSRLYSojo) if you want to see an even-more-animated version of this plot.

### Animated histogram

In [ ]:
px.histogram(gapminder,
            x = 'lifeExp',
            animation_frame = 'year',
            range_x = [20, 90],
            range_y = [0, 50],
            title = 'Distribution of Life Expectancy over Time')

### Choropleth

In [ ]:
px.choropleth(gapminder,
              locations = 'iso_alpha',
              color = 'lifeExp',
              hover_name = 'country',
              hover_data = {'iso_alpha': False},
              title = 'Life Expectancy Per Country',
              color_continuous_scale = px.colors.sequential.tempo
)

## Parting thoughts

### From Lecture 1: What is "data science"?

Data science is about **drawing useful conclusions from data using computation**. Throughout the quarter, we touched on several aspects of data science:

- In the first 4 weeks, we used Python to **explore** data.
    - Lots of visualization 📈📊 and "data manipulation", using industry-standard tools.

- In the next 4 weeks, we used data to **infer** about a population, given just a sample.
    - Rely heavily on simulation, rather than formulas.

- In the last 2 weeks, we used data from the past to **predict** what may happen in the future.
    - A taste of machine learning 🤖.

- In future courses – including DSC 20 and 40A, which you may be taking next quarter – you'll revisit all three of these aspects of data science.

### Thank you!

This course would not have been possible without...
- **Our 2 graduate TAs**: Dylan Stockard and Dasha Veraksa.
- **Our 17 undergraduate tutors**: Gabriel Cha, Eric Chen, Charlie Gillet, Vanessa Hu, Dylan Lee, Anthony Li, Jasmine Lo, Linda Long, Aishani Mohapatra, Harshita Saha, Abel Seyoum, Selim Shaalan, Yutian (Skylar) Shi, Tony Ta, Zairan Xiang, Diego Zavalza, and Lauren (Luran) Zhang.


- Learn [more about tutoring](https://datascience.ucsd.edu/academics/undergraduate/dsc-tutors/) – it's fun, and you can be a tutor as early as your 3rd quarter at UCSD!
- Keep in touch! [dsc10.com/staff](https://dsc10.com/staff)
    - After grades are released, we'll make a post on EdStem where you can ask course staff for advice on courses and UCSD more generally.

## Good luck on your finals! 🎉

### And see you tomorrow at 3PM in Galbraith Hall 242. ⏰